In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
df=pd.read_csv('clothing.csv')

In [3]:
# Scale rating feature to make its range between 0-5
df['rating'] = df['rating']/2

In [4]:
# Add new feature which has binary rating i.e either positive or negative 
# Categorise as 1 if rating is either 5 or 4 and rented item is not small or large in size
rating_to_binary = (df["rating"] == 5.0) | (df["rating"] == 4.0) | (df["rating"] == 3.0) & (df["fit"] == "fit")
rating_to_binary 
df["class_rating"] = np.where(rating_to_binary == True, 1, 0)
df["class_rating"]

0         1
1         1
2         1
3         1
4         1
         ..
192539    1
192540    1
192541    1
192542    1
192543    1
Name: class_rating, Length: 192544, dtype: int64

In [5]:
# Compressing records by considering the items which has more than 4 ratings and users who purchased more than 5 items
# Or give rating to more than 5 items


# including items with more than 4 ratings
itemRat_count = pd.DataFrame(df.groupby("item_id")["rating"].count())
itemRat_count = itemRat_count.loc[(itemRat_count["rating"] > 4)]
itemRat_count.reset_index(inplace=True)
#print('itemRat_count')
clothingRed = df[df["item_id"].isin(itemRat_count["item_id"])]
#print(clothingRed)

In [6]:
# including users with more than 4 purchases
userRat_count = pd.DataFrame(clothingRed.groupby("user_id")["rating"].count())
userRat_count = userRat_count.loc[(userRat_count.rating > 5)]
userRat_count.reset_index(inplace=True)
userRat_count.drop("rating", axis=1, inplace=True)
clothingRed = clothingRed[clothingRed["user_id"].isin(userRat_count["user_id"])]
clothingRed.reset_index(inplace=True, drop=True)

In [7]:
from sklearn.model_selection import train_test_split

# In train_test_split, user_id is stratified so same user id appears in the both the training and test test

clothing_train, clothing_test = train_test_split(clothingRed, stratify=clothingRed["user_id"],test_size=.2, random_state=1017)
print(clothing_train.shape)
print(clothing_test.shape)

(32272, 16)
(8069, 16)


In [20]:
clothing_test.groupby('user_id')['rating'].count().tail(50)

user_id
990235    3
990369    2
990405    2
990544    2
990625    1
990657    3
991099    3
991579    2
991702    3
991757    2
991902    1
991993    1
992004    1
992421    1
992560    1
992663    1
993019    1
993059    3
993344    1
993433    3
993618    1
993966    1
994205    1
994398    5
994448    2
994525    2
994767    1
994906    2
994907    3
995029    1
995412    5
995591    1
995826    8
995929    1
996183    2
996512    4
996592    1
997058    1
997189    2
997237    2
997360    3
998194    3
998267    1
998563    3
998780    1
998923    1
999183    2
999231    4
999274    1
999431    2
Name: rating, dtype: int64

# Content based recommendation using TF-IDF

In [9]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Remove useless words 
# Ref: https://towardsdatascience.com/content-based-recommender-systems-28a1dbd858f5
# Ref: https://github.com/ariellem2/Fit_Predictions_and_Recommendations/blob/master/Rent_the_Runway_Recommender.ipynb
stop_words = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/swapnilshevate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Add new column which has  the text data such as category, review summary, review text
# Using this since in the dataset there is no meta data given for items
# Hence creating meta data extracting most of the text data from features 

df["item_review_text"] = (df["category"] + " " + df["review_summary"] + " " + df["review_text"]).str.lower()
df["item_review_text"] = df["item_review_text"].astype(str)
df["item_review_text"]= df["item_review_text"].apply(lambda x: " ".join([word for word in x.split() if word not in (stop_words)]))


In [11]:
df[["item_id", "item_review_text"]].tail()

,item_id,item_review_text
192539,2252812,jumpsuit love it!!! first item im thinking buy...
192540,682043,dress love it! pattern contrast dress really s...
192541,683251,"dress loud patterning, flattering fit like dvf..."
192542,126335,dress loved dress comfortable photographed ama...
192543,127865,gown wore beautiful black tie optional wedding...


In [12]:
agg_item_text = pd.DataFrame(df.groupby("item_id")["item_review_text"].sum())
agg_item_text.head()

,item_review_text
item_id,
123373,gown beautiful black tie wedding dress beautif...
123793,gown stunning gown. wore heart ball received c...
124204,"dress opened rtr box saw sequins, swear heart ..."
124553,dress lovely wedding guest dress loved dress. ...
125424,dress wore rehearsal dinner. classy. dress arr...


In [13]:
tf_idf = TfidfVectorizer()
tf_idf_matrix = tf_idf.fit_transform(agg_item_text["item_review_text"])
item_tfidf_similarity = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
item_tfidf_similarity_df = pd.DataFrame(item_tfidf_similarity, columns=agg_item_text.index, index=agg_item_text.index)

# Create item-item similarity matrix usig td-idf
item_tfidf_similarity_df.head()

item_id,123373,123793,124204,124553,125424,125465,125564,126335,127081,127495,...,2960969,2961855,2962646,2963344,2963601,2963850,2964470,2965009,2965924,2966087
item_id,,,,,,,,,,,,,,,,,,,,,
123373,1.000000,0.920049,0.646582,0.683497,0.679712,0.920857,0.939897,0.681005,0.877858,0.685035,...,0.199865,0.038458,0.129807,0.116882,0.039445,0.133218,0.151984,0.060291,0.102984,0.170247
123793,0.920049,1.000000,0.712092,0.696588,0.689520,0.935512,0.935278,0.751240,0.889452,0.675245,...,0.184193,0.037216,0.134070,0.118367,0.031789,0.124774,0.135332,0.062114,0.094801,0.177672
124204,0.646582,0.712092,1.000000,0.852282,0.844124,0.651073,0.655472,0.930405,0.634690,0.866898,...,0.196270,0.035105,0.138122,0.144895,0.038251,0.129289,0.160755,0.063575,0.089681,0.127207
124553,0.683497,0.696588,0.852282,1.000000,0.923627,0.711417,0.687588,0.885807,0.666405,0.886473,...,0.240113,0.045744,0.158160,0.154095,0.058190,0.154087,0.185648,0.061639,0.114007,0.143805
125424,0.679712,0.689520,0.844124,0.923627,1.000000,0.676221,0.686581,0.876664,0.658333,0.875506,...,0.241356,0.049377,0.171951,0.152510,0.054273,0.152740,0.182974,0.061219,0.120921,0.140214


In [14]:
def evaluation(recommendations, user):
    # Get records of user from test data which has same user id as passed in the argument
    user_pref = clothing_test[clothing_test.user_id.isin([user])]
    #print(user_pref)
    # Get items from user test pref and check which items matches with calculated top 50 items using test data
    items_in_50 = user_pref[user_pref.item_id.isin(recommendations.item_id)]
    #print(items_in_50)
    
    # 'accuratelyCal_in50' is used for how many accurately calculated items out of top 50 similar ones 
    # Recall: (Total correctly predicted) / (total true positive + total false negative)
    recallCal = {"user_id": user,
                   "ratings_count": len(user_pref),
                   "accuratelyCal_in50": len(items_in_50[items_in_50.class_rating == 1]),
                   "recall_in50": len(items_in_50[items_in_50.class_rating == 1]) / len(user_pref)}
                    
    return recallCal
# evaluatio = pd.DataFrame()

In [15]:
def tfidf_contentRecSys(id):
    
    # Get record of user from train data which has same user id as passed in the argument
    top_train_user_item = clothing_train[clothing_train.user_id == id][:1]
    # print(top_train_user_item)
    # To get item id corresponding to above record
    top_train_user_item = int(top_train_user_item.item_id)
    # print(top_train_user_item)
    # To compute all the similar items corresponding to above record's item
    item_similarities = item_tfidf_similarity_df[[top_train_user_item]].sort_values([top_train_user_item], ascending=False)
    # print(item_similarities)
    # Take top 50 simialr items
    top_50 = item_similarities.iloc[1:51]
    # print(top_50)
    top_50.reset_index(inplace=True)
    # print(top_50)               
    return recommender_test_eval(recommendations = top_50, user = id)



test_user = 2927 
tfidf_contentRecSys(test_user)

{'user_id': 2927,
 'ratings_count': 3,
 'accuratelyCal_in50': 1,
 'recall_in50': 0.3333333333333333}

In [22]:
test_user = 998563
tfidf_contentRecSys(test_user)

{'user_id': 998563,
 'ratings_count': 3,
 'accuratelyCal_in50': 1,
 'recall_in50': 0.3333333333333333}

In [26]:
test_user = 991757
tfidf_contentRecSys(test_user)

{'user_id': 991757,
 'ratings_count': 2,
 'accuratelyCal_in50': 1,
 'recall_in50': 0.5}